In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from collections import defaultdict
from my_functions import regular_comp

# Combining datasets

I kept the Steam appids in all my datasets to ensure I had easy way to combine them together

In [39]:
#importing seperate datasets
games = pd.read_csv("../data/steam_clean.csv")
discounts = pd.read_csv("../data/summer_sale_19_v2.csv")
players = pd.read_csv("../data/steam_player_nos_v2.csv")

In [40]:
#checking sizes before and after dropping nans
print(games.shape)
print(discounts.shape)
print(players.shape)

(22579, 18)
(22579, 3)
(22579, 9)


In [41]:
print(games.dropna().shape)
print(discounts.dropna().shape)
print(players.dropna().shape)

(22579, 18)
(21052, 3)
(14536, 9)


In [42]:
#combining them together
steam_df = games.merge(discounts, on=["appid", "name"])
steam_df = steam_df.merge(players, on=["appid", "name"])
steam_df.shape

(22579, 26)

In [43]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22579 entries, 0 to 22578
Data columns (total 26 columns):
appid               22579 non-null int64
name                22579 non-null object
release_date        22579 non-null object
english             22579 non-null int64
developer           22579 non-null object
publisher           22579 non-null object
platforms           22579 non-null object
required_age        22579 non-null int64
categories          22579 non-null object
genres              22579 non-null object
steamspy_tags       22579 non-null object
achievements        22579 non-null int64
positive_ratings    22579 non-null int64
negative_ratings    22579 non-null int64
average_playtime    22579 non-null int64
median_playtime     22579 non-null int64
owners              22579 non-null object
price               22579 non-null float64
discount            21052 non-null object
all_time_peak       20096 non-null float64
jul_19_av           17973 non-null float64
jul_19_peak   

In [44]:
steam_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,owners,price,discount,all_time_peak,jul_19_av,jul_19_peak,jun_19_av,jun_19_peak,may_19_av,may_19_peak
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,10000000-20000000,7.19,0,65188.0,9317.42,15309.0,9165.61,14925.0,9246.58,16503.0
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,5000000-10000000,3.99,90%,191.0,68.37,144.0,64.79,130.0,58.06,142.0
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,5000000-10000000,3.99,90%,654.0,99.35,157.0,99.19,188.0,92.56,193.0
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,5000000-10000000,3.99,90%,627.0,8.55,25.0,6.47,20.0,5.88,48.0
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,5000000-10000000,3.99,0,237.0,112.87,230.0,73.91,217.0,52.45,129.0


In [33]:
#Various column engineerings:

#Dopping all nans (i.e. where I couln't find sale info or player counts)
steam_df = steam_df.dropna()
steam_df.reset_index(inplace=True)

#converting the discount to a float, and creating a binary variable
steam_df["discount"] = steam_df["discount"].map(lambda x: float(x.replace("%", "") if x !=0 else x))
steam_df["discount_bin"] = steam_df["discount"].map(lambda x: 1 if x > 0 else 0)

#Converting the age to a string as it's categorical
steam_df["required_age"]=steam_df["required_age"].map(lambda x: str(x))

#Changing release date to datetime
steam_df["release_date"] = pd.to_datetime(steam_df["release_date"])

#Changing achievements to a yes/no category
steam_df["achievements"] = steam_df["achievements"].map(lambda x: 1 if x > 0 else 0)

#Removing SteamSpy conjecture
steam_df.drop(columns=["average_playtime", "median_playtime", "owners"], inplace=True)

#Steamspy tags preovide no extra info that the cats/genres columns don't already provide
steam_df.drop(columns="steamspy_tags", inplace=True)

#Getting percentage of positive reviews, and total number of reviews
steam_df["ratingpc"] = steam_df["positive_ratings"]/(steam_df["positive_ratings"]+steam_df["negative_ratings"])
steam_df["total_ratings"]=steam_df["negative_ratings"]+steam_df["positive_ratings"]
steam_df.drop(columns=["positive_ratings", "negative_ratings"], inplace=True)


#Creating platform categories
steam_df["windows"] = steam_df["platforms"].map(lambda x: 1 if "windows" in x else 0)
steam_df["linux"] = steam_df["platforms"].map(lambda x: 1 if "linux" in x else 0)
steam_df["mac"] = steam_df["platforms"].map(lambda x: 1 if "mac" in x else 0)

steam_df.drop(columns="platforms", inplace=True)

#Creating categories of Categories and Genres
categories = {"appid" : []}

#creating a dictionary key for each category
for cell in steam_df["categories"]:
    for cat in cell.split(";"):
        categories["cat_" + cat]=[]

#for each game, check if a game has it and append a 1 or 0 to the dictionary key accordingly
for i in range(0, len(steam_df)):
    categories["appid"].append(steam_df["appid"][i])
    for cat in [key for key in categories.keys() if key != "appid"]:
        if (cat[4:]) in steam_df["categories"][i]:
            categories[cat].append(1)
        else:
            categories[cat].append(0)
            
#do the same for genres            
genres = {"appid" : []}

for cell in steam_df["genres"]:
    for gen in cell.split(";"):
        genres["gen_" + gen]=[]

for i in range(0, len(steam_df)):
    genres["appid"].append(steam_df["appid"][i])
    for gen in [key for key in genres.keys() if key != "appid"]:
        if (gen[4:]) in steam_df["genres"][i]:
            genres[gen].append(1)
        else:
            genres[gen].append(0)
            
#add them into the dataframe
steam_df = steam_df.merge(pd.DataFrame(categories), on="appid")
steam_df = steam_df.merge(pd.DataFrame(genres), on="appid")

steam_df.drop(columns =["categories", "genres"], inplace=True)

#dropping the categories and genres that are too small
for col in steam_df.columns:
    if steam_df[col].dtype == "int64":
        if steam_df[col].sum() < 700:
             steam_df.drop(columns=col, inplace=True)


steam_df.shape

(13581, 45)

In [34]:
#Computing potential target columns

#uplift between jul and jun averages, both absolute and realative
steam_df["jun_jul_avuplift"] = steam_df["jul_19_av"]-steam_df["jun_19_av"]
steam_df["jun_jul_avupliftpc"] = steam_df["jun_jul_avuplift"]/steam_df["jun_19_av"]

#same for may - jun as this may be a useful variable
steam_df["may_jun_avuplift"] = steam_df["jun_19_av"]-steam_df["may_19_av"]
steam_df["may_jun_avupliftpc"] = steam_df["may_jun_avuplift"]/steam_df["may_19_av"]
steam_df["any_jun_uplift"] = steam_df["may_jun_avuplift"].map(lambda x: 1 if x>0 else 0)

#dropping jul peak column
steam_df.drop(columns=["jul_19_peak"], inplace=True)


In [35]:
steam_df.columns

Index(['index', 'appid', 'name', 'release_date', 'english', 'developer',
       'publisher', 'required_age', 'achievements', 'price', 'discount',
       'all_time_peak', 'jul_19_av', 'jun_19_av', 'jun_19_peak', 'may_19_av',
       'may_19_peak', 'discount_bin', 'ratingpc', 'total_ratings', 'windows',
       'linux', 'mac', 'cat_Multi-player', 'cat_Online Multi-Player',
       'cat_Local Multi-Player', 'cat_Single-player', 'cat_Steam Cloud',
       'cat_Steam Achievements', 'cat_Steam Trading Cards',
       'cat_Partial Controller Support', 'cat_Stats', 'cat_Co-op',
       'cat_Full controller support', 'cat_Steam Leaderboards',
       'cat_Shared/Split Screen', 'gen_Action', 'gen_Adventure', 'gen_Indie',
       'gen_Strategy', 'gen_RPG', 'gen_Simulation', 'gen_Casual',
       'gen_Early Access', 'jun_jul_avuplift', 'jun_jul_avupliftpc',
       'may_jun_avuplift', 'may_jun_avupliftpc', 'any_jun_uplift'],
      dtype='object')

In [36]:
#reordering columns
steam_df = steam_df[['index', 'appid', 'name', 'release_date', 'english', 'developer',
       'publisher', 'required_age', 'achievements', 'total_ratings', 'ratingpc', 'price', 
       'discount', 'discount_bin', 'windows', 'linux', 'mac', 'cat_Multi-player',
       'cat_Online Multi-Player', 'cat_Single-player', 'cat_Steam Cloud',
       'cat_Steam Achievements', 'cat_Steam Trading Cards',
       'cat_Partial Controller Support', 'cat_Stats', 'cat_Co-op',
       'cat_Full controller support', 'cat_Steam Leaderboards',
       'cat_Shared/Split Screen', 'gen_Action', 'gen_Adventure', 'gen_Indie',
       'gen_Strategy', 'gen_RPG', 'gen_Simulation', 'gen_Casual',
       'gen_Early Access', 'all_time_peak', 'may_19_av', 'may_19_peak', 'jun_19_av', 
       'jun_19_peak', 'jul_19_av', 'may_jun_avuplift', 'may_jun_avupliftpc',
       'jun_jul_avuplift', 'jun_jul_avupliftpc']]

In [37]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13581 entries, 0 to 13580
Data columns (total 47 columns):
index                             13581 non-null int64
appid                             13581 non-null int64
name                              13581 non-null object
release_date                      13581 non-null datetime64[ns]
english                           13581 non-null int64
developer                         13581 non-null object
publisher                         13581 non-null object
required_age                      13581 non-null object
achievements                      13581 non-null int64
total_ratings                     13581 non-null int64
ratingpc                          13581 non-null float64
price                             13581 non-null float64
discount                          13581 non-null float64
discount_bin                      13581 non-null int64
windows                           13581 non-null int64
linux                             13581 non-nul

In [38]:
steam_df.to_csv("../data/steam_munged_maydata.csv", index=False)